## Classical models prediction

Попробуем предсказать класс новостного заголовка с помощью классических моделей

### Imports

In [1]:
import pandas as pd
import numpy as np
from typing import Union

from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

RND_STATE = 3010
np.random.seed(RND_STATE)

## Data loading

In [2]:
train_ds = pd.read_csv('data/c_train.csv')
test_ds = pd.read_csv('data/c_test.csv')

In [3]:
train_ds.head(3)

,cleaned,is_fake
0,москвич владимир клутин приходить счет вмешате...,1
1,агент кокорин называть езда встречок житейский...,0
2,госдума рассматривать возможность введение сек...,1


In [4]:
test_ds.head()

,cleaned,is_fake
0,роскомнадзор представлять реестр сочетание цве...,0
1,ночь минск президентский горе беларашмор пик д...,0
2,бывший спичрайтер юрий лоза рассказывать трудн...,0
3,сельский церковь собирать рекордно низкий коли...,0
4,акция google рухнуть объявление перезапуск rutube,0


## Feature Engineering
Добавим сюда признаки, полученные в ходе EDA

In [5]:
def generate_features(ds: pd.DataFrame) -> pd.DataFrame:
    result = ds.copy()
    result['word_count'] = [len(t.split(' ')) for t in result.cleaned]
    
    return result

In [6]:
train_ds = generate_features(train_ds)
test_ds = generate_features(test_ds)

Переведём данные в векторную форму с помощью TF-IDF

In [7]:
usefull_columns = ['word_count']

vector_pipeline = ColumnTransformer([('tf_idf_vect', TfidfVectorizer(), 'cleaned'), 
                                     ('selector', 'passthrough', usefull_columns)], )

## Подготовка тренировки

Для того, чтобы проверить качество модели более объективно, выделим часть данных из тренировочной выборки для валидации 

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(train_ds.drop(columns='is_fake'), train_ds.is_fake, 
                                                      test_size=0.2, random_state=RND_STATE, stratify=train_ds.is_fake)

In [9]:
x_train

,cleaned,word_count
726,европа отступать банановый фронт,4
2660,роскомнадзор проверять пропаганда гомосексуали...,9
7,россиянин обхитрить рост цена,4
1733,интер рао выкупать огк норникель,5
4658,главврач нок рб тимановский весь признак вялот...,11
...,...,...
43,онлайн свадьба год россия зафиксировать случай...,7
215,сколково доставлять образец кристалл комета га...,9
1482,бузов приглашать фанат встречаться январь цент...,7
680,боксер дрозд оценивать перспектива поветкин пр...,7


In [10]:
def print_results(cv_results: list) -> None:
    print()
    
    for ind, val in enumerate(cv_results, 1):
        print(f'\t{ind} -> {round(val, 3)}')
        
    print('\nСреднее значение метрики:', round(np.mean(cv_results), 3))

    
def print_grid_results(grid_searcher: Union[GridSearchCV, RandomizedSearchCV]) -> None:
    print("Результаты подбора параметров:\n")
    print("Лучшие параметры:\n")

    for param_name, param_value in grid_searcher.best_params_.items():
        print(f'\t{param_name} = {param_value}')

    print("\nКачество модели на этих параметрах:", round(grid_searcher.best_score_, 3))

## RandomForest

Обучим случайный лес, который задаст нам baseline для дальнейшего обучения. Данная модель почти не требует настройки и прекрасно работает из коробки, поэтому она хорошо подойдёт для начала

In [11]:
rf_model = RandomForestClassifier(n_estimators=250, random_state=RND_STATE)

rf_pipeline = make_pipeline(vector_pipeline, rf_model)

In [12]:
%%time

rf_cv_res = cross_val_score(rf_pipeline, x_train, y_train, 
                            cv=5,
                            scoring=make_scorer(f1_score),
                            n_jobs=-1,
                            error_score='raise',
                           )

Wall time: 9.34 s


In [13]:
print("Результаты тренировки случайного леса")
print_results(rf_cv_res)

Результаты тренировки случайного леса

	1 -> 0.792
	2 -> 0.804
	3 -> 0.806
	4 -> 0.81
	5 -> 0.787

Среднее значение метрики: 0.8


Попробуем подобрать параметры, чтобы немного улучшить результат

In [14]:
rf_tuned_model = RandomForestClassifier(n_estimators=250, n_jobs=-1, random_state=RND_STATE)

rf_tuned_pipeline = make_pipeline(vector_pipeline, rf_tuned_model)

In [15]:
rf_param_grid = {
    "randomforestclassifier__criterion": ['gini', 'entropy'],
    "randomforestclassifier__max_features": ['auto', 'sqrt', 'log2'],
    "randomforestclassifier__min_samples_leaf": [1, 2, 3, 5, 8],
}

rf_grid_searcher = GridSearchCV(rf_tuned_pipeline, rf_param_grid, 
                                cv=5, 
                                n_jobs=-1, 
                                scoring=make_scorer(f1_score)
                               )

%time rf_grid_searcher.fit(x_train, y_train);
print("Тренировка окончена")

Wall time: 1min 3s
Тренировка окончена


In [16]:
print_grid_results(rf_grid_searcher)

Результаты подбора параметров:

Лучшие параметры:

	randomforestclassifier__criterion = entropy
	randomforestclassifier__max_features = log2
	randomforestclassifier__min_samples_leaf = 1

Качество модели на этих параметрах: 0.828


## Logistic regression

Логистическая регрессия часто используется в задачах классификации, а в нашем случае, когда размерность пространства признаков может расти очень быстро при использовании биграм, триграм и тд, данная модель будет быстро обучаться и работать

In [17]:
log_reg_model = LogisticRegression()

log_reg_pipeline = make_pipeline(vector_pipeline, log_reg_model)

In [18]:
%%time
log_reg_cv_res = cross_val_score(log_reg_pipeline, x_train, y_train,
                                 cv=5,
                                 scoring=make_scorer(f1_score),
                                 n_jobs=-1,
                                 error_score='raise',
                                )

Wall time: 666 ms


In [19]:
print("Результаты тренировки логистической регрессии")
print_results(log_reg_cv_res)

Результаты тренировки логистической регрессии

	1 -> 0.809
	2 -> 0.834
	3 -> 0.831
	4 -> 0.83
	5 -> 0.826

Среднее значение метрики: 0.826


Настраиваем модель с помощью гиперпараметров

Так как мы работаем с логистической регрессией нужно не забыть про масштабирование признаков(хотя tf-idf уже упростил эту задачу)

In [20]:
log_reg_tuned_model = LogisticRegression(solver='liblinear', random_state=RND_STATE, max_iter=400)

log_reg_tuned_pipeline = make_pipeline(vector_pipeline, StandardScaler(with_mean=False), log_reg_tuned_model)

In [21]:
log_reg_param_grid = {
    "logisticregression__C": [1e-5, 1e-4, 1e-3, 1e-2, 7e-2, 8e-2, 9e-2, 1e-1, 2e-1, 3e-1, 4e-1, 5e-1, 1, 1e1, 1e2],
    "logisticregression__penalty": ['l2', 'l1'],
}

log_reg_grid_searcher = GridSearchCV(log_reg_tuned_pipeline, log_reg_param_grid, 
                                     cv=5, 
                                     n_jobs=-1, 
                                     scoring=make_scorer(f1_score),
                                    )

%time log_reg_grid_searcher.fit(x_train, y_train);
print("Тренировка окончена")

Wall time: 5.65 s
Тренировка окончена


In [22]:
print_grid_results(log_reg_grid_searcher)

Результаты подбора параметров:

Лучшие параметры:

	logisticregression__C = 0.001
	logisticregression__penalty = l2

Качество модели на этих параметрах: 0.838


## GBM (Gradient Boosting Machine)

Данный алгоритм активно используется на Kaggle соревнованиях и является очень мощной моделью, которая при правильной настройке может выдать очень неплохой результат

In [23]:
gbm_model = XGBClassifier()

gbm_pipeline = make_pipeline(vector_pipeline, StandardScaler(with_mean=False), gbm_model)

In [24]:
%%time
gbm_cv_res = cross_val_score(gbm_pipeline, x_train, y_train,
                             cv=5,
                             scoring=make_scorer(f1_score),
                             n_jobs=-1,
                             error_score='raise',
                            )

Wall time: 3.07 s


In [25]:
print("Результаты тренировки градиетного бустинга")
print_results(gbm_cv_res)

Результаты тренировки градиетного бустинга

	1 -> 0.782
	2 -> 0.806
	3 -> 0.795
	4 -> 0.803
	5 -> 0.794

Среднее значение метрики: 0.796


А теперь проведём настройку параметров

In [26]:
gbm_tuned_model = XGBClassifier(use_label_encoder=False)

gbm_tuned_pipeline = make_pipeline(vector_pipeline, StandardScaler(with_mean=False), gbm_tuned_model)

In [27]:
gbm_param_grid = {
    "xgbclassifier__n_estimators": list(range(100, 501, 50)),
    "xgbclassifier__max_depth": [1, 2, 3, 5, 8],
    "xgbclassifier__learning_rate": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2],
    "xgbclassifier__reg_alpha": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2],
    "xgbclassifier__reg_lambda": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2],
}

gbm_grid_searcher = RandomizedSearchCV(gbm_tuned_pipeline, gbm_param_grid, 
                                       cv=5, 
                                       n_jobs=-1, 
                                       scoring=make_scorer(f1_score),
                                       random_state=RND_STATE,
                                       n_iter=20,
                                      )

%time gbm_grid_searcher.fit(x_train, y_train);
print("Тренировка окончена")

[00:11:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 2min 7s
Тренировка окончена


In [28]:
print_grid_results(gbm_grid_searcher)

Результаты подбора параметров:

Лучшие параметры:

	xgbclassifier__reg_lambda = 10.0
	xgbclassifier__reg_alpha = 0.01
	xgbclassifier__n_estimators = 350
	xgbclassifier__max_depth = 3
	xgbclassifier__learning_rate = 1

Качество модели на этих параметрах: 0.804


Подбор параметров не очень эффективен, тк перебор по сетке целиком дорогой по времени, а случайный перебор не может подобрать хорошие параметры, тк видит только подмножество параметров. Воспользуемся ручным подбором параметров

In [29]:
gbm_params = {
    "xgbclassifier__n_estimators": 175,
    "xgbclassifier__max_depth": 4,
    "xgbclassifier__learning_rate": 0.6,
    "xgbclassifier__reg_alpha": 0.2,
    "xgbclassifier__reg_lambda": 4e-1,
    "xgbclassifier__colsample_bytree": 0.1,
}

gbm_tuned_pipeline.set_params(**gbm_params);

In [30]:
%%time 
gbm_tuned_cv_res = cross_val_score(gbm_tuned_pipeline, x_train, y_train,
                                   cv=5,
                                   scoring=make_scorer(f1_score),
                                   n_jobs=-1,
                                   error_score='raise',
                                  )

Wall time: 3.02 s


In [31]:
print("Результаты тренировки:")
print_results(gbm_tuned_cv_res)

Результаты тренировки:

	1 -> 0.798
	2 -> 0.806
	3 -> 0.804
	4 -> 0.804
	5 -> 0.802

Среднее значение метрики: 0.803


Подбор параметров в ручную не дал сильной прибавки в качестве

## Bagging

Как видно из предыдущего исследования лучше всего себя показала **логистическая регрессия**. Можно попробовать *объединить несколько логистических регрессий в один ансамбль* алгоритмов с помощью бэггинга. Возможно, полученная модель будет сильнее одного классификатора.

In [32]:
bagging_model = BaggingClassifier(LogisticRegression())

bagging_pipeline = make_pipeline(vector_pipeline, StandardScaler(with_mean=False), bagging_model)

In [33]:
%%time
bagging_cv_res = cross_val_score(bagging_pipeline, x_train, y_train,
                                 cv=5,
                                 scoring=make_scorer(f1_score),
                                 n_jobs=-1,
                                 error_score='raise',
                                )

Wall time: 2.09 s


In [34]:
print("Результаты тренировки бэггинга на логистической регрессии")
print_results(bagging_cv_res)

Результаты тренировки бэггинга на логистической регрессии

	1 -> 0.836
	2 -> 0.836
	3 -> 0.842
	4 -> 0.849
	5 -> 0.854

Среднее значение метрики: 0.843


Подберём параметры для бэггинга и логистического регрессора

In [35]:
bagging_tuned_model = BaggingClassifier(LogisticRegression(), #log_reg_grid_searcher.best_estimator_.named_steps['logisticregression'], 
                                        n_jobs=-1,
                                        random_state=RND_STATE,
                                       )

bagging_tuned_pipeline = make_pipeline(vector_pipeline, StandardScaler(with_mean=False), bagging_tuned_model)

In [36]:
bagging_param_grid = {
    "baggingclassifier__n_estimators": list(range(10, 101, 10)) + [3, 5, 8],
    "baggingclassifier__max_samples": [0.5, 0.7, 0.9, 1.0],
    "baggingclassifier__max_features": [0.5, 0.7, 0.9, 1.0],
}

bagging_grid_searcher = RandomizedSearchCV(bagging_tuned_pipeline, bagging_param_grid, 
                                           cv=5, 
                                           n_jobs=-1, 
                                           scoring=make_scorer(f1_score),
                                           random_state=RND_STATE,
                                           n_iter=20,
                                          )

%time bagging_grid_searcher.fit(x_train, y_train);
print("Тренировка окончена")

Wall time: 3min 28s
Тренировка окончена


In [37]:
print_grid_results(bagging_grid_searcher)

Результаты подбора параметров:

Лучшие параметры:

	baggingclassifier__n_estimators = 90
	baggingclassifier__max_samples = 0.7
	baggingclassifier__max_features = 0.5

Качество модели на этих параметрах: 0.856


Отсюда видно, что качество ансамбля моделей лучше чем один классификатор

## Проверка качества

Выполним проверку модели с помощью отложенной выборки, которую мы выделили в начале исследования

In [38]:
best_model = bagging_grid_searcher.best_estimator_

In [39]:
print("Качество лучшей модели на отложенной выборке: ")
print("F1 score ->", round(f1_score(y_valid, best_model.predict(x_valid)), 3))

Качество лучшей модели на отложенной выборке: 
F1 score -> 0.873


## Вывод

Лучшей моделью оказался *бэггинг на логистической регрессии* и лучшее качество, которого удалось достичь с помощью *классического подхода*, на отложенной выборке равно **0.873**

## Сохраняем предсказания для тестовой выборки

In [40]:
test_pred = best_model.predict(test_ds.drop(columns='is_fake'))

In [41]:
test_ds = pd.read_csv('data/dataset/test.tsv', sep='\t')

In [42]:
test_ds['is_fake'] = test_pred

In [43]:
test_ds.to_csv('predictions.tsv', sep='\t', index=False)